# Purpose

Figure out how to take an Openbabel molecule object and generate a set of minimum dihedral angles.

# Method

Starting with an exhaustive set of dihedrals, remove the duplicates s.t. there is only one dihedral per rotatable bond. Therefore, this produces one dihedral for each b,c in a a,b,c,d dihedral angle. The identities of a and d don't matter, but it may be preferential to use the indices for heavier atoms such as C,O instead of H.

In [1]:
from kaplan.inputs import Inputs

RDKit WARNING: [20:36:33] Enabling RDKit 2019.09.1 jupyter extensions


In [2]:
inputs = Inputs()
inputs.update_inputs({"struct_input": "butane"})

Selecting dihedral angles.
Done with inputs setup.


In [3]:
m = inputs.obmol

In [4]:
m

<openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x7f1b66631c60> >

In [5]:
print(m)

<openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x7f1b66631c60> >


In [10]:
from kaplan.geometry import get_angles, get_torsions

In [11]:
torsions = get_torsions(m)

In [12]:
torsions


[(8, 2, 0, 4, 1.068404989474897),
 (8, 2, 0, 1, -1.0526592728214157),
 (8, 2, 0, 5, 3.1082217953807962),
 (10, 2, 0, 4, -1.0215658602311113),
 (10, 2, 0, 1, 3.1405551846521624),
 (10, 2, 0, 5, 1.018250945674788),
 (9, 2, 0, 4, -3.111413345845887),
 (9, 2, 0, 1, 1.050707699037387),
 (9, 2, 0, 5, -1.0715965399399872),
 (11, 3, 1, 7, 1.0684979814362843),
 (11, 3, 1, 0, -1.0525729413005291),
 (11, 3, 1, 6, 3.1081898282106732),
 (12, 3, 1, 7, -1.0214598129925698),
 (12, 3, 1, 0, 3.1406545714502028),
 (12, 3, 1, 6, 1.0182320337818191),
 (13, 3, 1, 7, -3.1113304690556447),
 (13, 3, 1, 0, 1.0507839153871283),
 (13, 3, 1, 6, -1.0716386222812555),
 (4, 0, 1, 7, -1.1045851139024612),
 (4, 0, 1, 3, 1.00849296238523),
 (4, 0, 1, 6, 3.124335083296512),
 (2, 0, 1, 7, 1.0084377397768836),
 (2, 0, 1, 3, 3.1215158160645746),
 (2, 0, 1, 6, -1.04582737020373),
 (5, 0, 1, 7, 3.124217773957886),
 (5, 0, 1, 3, -1.045889456934009),
 (5, 0, 1, 6, 1.0699526639772732)]

In [13]:
print(inputs.diheds)

[(2, 0, 1, 3), (1, 0, 2, 10), (0, 1, 3, 13)]


In [14]:
from kaplan.tools import plot_3d

In [16]:
plot_3d()

In [19]:
unique_bc = []  # list of tuples of atom indices
connected_ad = []

for dihed in torsions:
    if dihed[1:3] not in unique_bc and dihed[2:0:-1] not in unique_bc:
        unique_bc.append(dihed[1:3])
print(unique_bc)

[(2, 0), (3, 1), (0, 1)]


In [36]:
atomic_num_sums = []
for i, dihed in enumerate(torsions):
    atomic_num_sums.append((sum([inputs.atomic_nums[dihed[0]], inputs.atomic_nums[dihed[1]],
                               inputs.atomic_nums[dihed[2]], inputs.atomic_nums[dihed[3]]]), i))
atomic_num_sums = sorted(atomic_num_sums, reverse=True)

In [37]:
torsions_by_mass = []
for i, torsion in enumerate(torsions):
    torsions_by_mass.append(torsions[atomic_num_sums[i][1]])
for t in torsions_by_mass:
    print(t)

(2, 0, 1, 3, 3.1215158160645746)
(5, 0, 1, 3, -1.045889456934009)
(2, 0, 1, 6, -1.04582737020373)
(2, 0, 1, 7, 1.0084377397768836)
(4, 0, 1, 3, 1.00849296238523)
(13, 3, 1, 0, 1.0507839153871283)
(12, 3, 1, 0, 3.1406545714502028)
(11, 3, 1, 0, -1.0525729413005291)
(9, 2, 0, 1, 1.050707699037387)
(10, 2, 0, 1, 3.1405551846521624)
(8, 2, 0, 1, -1.0526592728214157)
(5, 0, 1, 6, 1.0699526639772732)
(5, 0, 1, 7, 3.124217773957886)
(4, 0, 1, 6, 3.124335083296512)
(4, 0, 1, 7, -1.1045851139024612)
(13, 3, 1, 6, -1.0716386222812555)
(13, 3, 1, 7, -3.1113304690556447)
(12, 3, 1, 6, 1.0182320337818191)
(12, 3, 1, 7, -1.0214598129925698)
(11, 3, 1, 6, 3.1081898282106732)
(11, 3, 1, 7, 1.0684979814362843)
(9, 2, 0, 5, -1.0715965399399872)
(9, 2, 0, 4, -3.111413345845887)
(10, 2, 0, 5, 1.018250945674788)
(10, 2, 0, 4, -1.0215658602311113)
(8, 2, 0, 5, 3.1082217953807962)
(8, 2, 0, 4, 1.068404989474897)


In [38]:
min_diheds_new = []
unique_bc = []
for dihed in torsions_by_mass:
    if dihed[1:3] not in unique_bc and dihed[2:0:-1] not in unique_bc:
        unique_bc.append(dihed[1:3])
        min_diheds_new.append(dihed[:4])
print(unique_bc)
print(min_diheds_new)

[(0, 1), (3, 1), (2, 0)]
[(2, 0, 1, 3), (13, 3, 1, 0), (9, 2, 0, 1)]


In [41]:
def filter_duplicate_diheds(dihedrals_list, atomic_nums):
    """Remove duplicate dihedrals based on b-c rotatable bond.
    
    Parameters
    ----------
    dihedrals_list : list(tuple(int, int, int, int, float))
        All possible dihedrals for a molecule.
    atomic_nums : list(int)
        Atomic numbers in order of index, 0-based.
        For example: [1,1,6] means 0 H, 1 H, 2 C.
    
    Returns
    -------
    list(tuple(int, int, int, int))
        Unique dihedrals with priority given to
        highest atomic number sum.
    
    """
    atomic_num_sums = []
    for i, dihed in enumerate(dihedrals_list):
        atomic_num_sums.append((sum([atomic_nums[dihed[0]], atomic_nums[dihed[1]],
                                     atomic_nums[dihed[2]], atomic_nums[dihed[3]]]), i))
    atomic_num_sums = sorted(atomic_num_sums, reverse=True)
    
    dihedrals_by_mass = []
    for i, dihed in enumerate(dihedrals_list):
        dihedrals_by_mass.append(dihedrals_list[atomic_num_sums[i][1]])
    
    min_diheds_new = []
    unique_bc = []
    for dihed in dihedrals_by_mass:
        if dihed[1:3] not in unique_bc and dihed[2:0:-1] not in unique_bc:
            unique_bc.append(dihed[1:3])
            min_diheds_new.append(dihed[:4])
    return min_diheds_new

In [42]:
result = filter_duplicate_diheds(torsions, inputs.atomic_nums)

In [43]:
result

[(2, 0, 1, 3), (13, 3, 1, 0), (9, 2, 0, 1)]